In [24]:
import pandas as pd

In [46]:
text = pd.read_csv('data/text.tsv', delimiter='\t', header=None, names=["sequence", "text"])
print(text.shape)
text.tail(30)

(19165, 2)


,sequence,text
19135,19136,petrie stores corp pst> sets quarterly. qtly d...
19136,19137,morocco secures bulk of extra aid sought. moro...
19137,19138,desoto dso> sells industrial chemical assets. ...
19138,19139,"data tapes erased in vw fraud, chairman says. ..."
19139,19140,"imf, development banks, to assist ecuador. it ..."
19140,19141,safeguard health sfgd> 4th qtr loss. shr loss ...
19141,19142,rubber talks chairman cites slight progress. t...
19142,19143,cummins cum>recalls workers on signs of upturn...
19143,19144,"britannia"" coin to sell at premium. the new br..."
19144,19152,Sat Feb 4 15:49:44.061 <airportd[69]> ERROR: ...


In [48]:
tags = pd.read_csv('data/oldfiles/tags.tsv', delimiter='\t', header=None, names=["sequence", "tags"])
print(tags.shape)
tags[19136:19175]

(19737, 2)


,sequence,tags
19136,19137,"morocco,worldbank"
19137,19138,"acq,usa"
19138,19139,west-germany
19139,19140,"usa,ecuador,imf"
19140,19141,"earn,usa"
19141,19142,"rubber,switzerland,inro"
19142,19143,usa
19143,19144,"gold,uk"
19144,19145,"acq,usa,sweden"
19145,19146,"interest,canada"


In [29]:
def tokenisetags(multitags):
    return tuple(set(multitags.split(',')))

text = text.append(hippa_text)
tags = tags.append(hippa_tag)
tags.tags = tags.tags.apply(tokenisetags)
text.tail()
tags.tail()

,sequence,tags
5,19722.0,"(hippa,)"
6,19723.0,"(hippa,)"
7,19724.0,"(hippa,)"
8,19725.0,"(hippa,)"
9,19726.0,"(hippa,)"


In [30]:
len(tags)

19747

In [31]:
len(text)

19175

In [32]:
from sklearn.preprocessing import MultiLabelBinarizer
lb = MultiLabelBinarizer()
y = lb.fit_transform(tags.tags)


In [33]:
import numpy as np
msk = np.random.rand(len(text)) < 0.8
#N = int(.8 * text.text.shape[0])
Xtrain = text.text[msk]
Xtest = text.text[~msk]
ytr = y[msk]
yte = y[~msk]

/home/canwill/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 19747 but corresponding boolean dimension is 19175
/home/canwill/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 19747 but corresponding boolean dimension is 19175


In [37]:
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import re

cachedStopWords = stopwords.words("english")

def tokenize(text):
    min_length = 3
    words = map(lambda word: word.lower(), word_tokenize(text))
    words = [word for word in words if word not in cachedStopWords]
    tokens = (list(map(lambda token: PorterStemmer().stem(token),
                                   words)))
    p = re.compile('[a-zA-Z]+');
    filtered_tokens =    list(filter (lambda token: p.match(token) and
                               len(token) >= min_length,
                               tokens))
    return filtered_tokens


In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Tokenisation
stop_words = stopwords.words("english")
vectorizer = TfidfVectorizer(stop_words=stop_words,
                             tokenizer=tokenize)

In [39]:
%%time
Xtr = vectorizer.fit_transform(Xtrain)
print(Xtr.shape)

(15404, 33666)
CPU times: user 1min 1s, sys: 136 ms, total: 1min 1s
Wall time: 1min 1s


In [40]:
%%time
Xte = vectorizer.transform(Xtest)
print(Xte.shape)

(3771, 33666)
CPU times: user 14.9 s, sys: 12 ms, total: 14.9 s
Wall time: 14.9 s


### SVC

In [41]:
%%time
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier

classifier = OneVsRestClassifier(LinearSVC(random_state=42))
classifier.fit(Xtr, ytr)

/home/canwill/anaconda2/lib/python2.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 10 is present in all training examples.
  str(classes[c]))
/home/canwill/anaconda2/lib/python2.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 22 is present in all training examples.
  str(classes[c]))
/home/canwill/anaconda2/lib/python2.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 55 is present in all training examples.
  str(classes[c]))
/home/canwill/anaconda2/lib/python2.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 74 is present in all training examples.
  str(classes[c]))
/home/canwill/anaconda2/lib/python2.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 120 is present in all training examples.
  str(classes[c]))
/home/canwill/anaconda2/lib/python2.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 124 is present in all training examples.
  str(classes[c]))
/home/canwill/anaconda2/lib/python2.7/

CPU times: user 20.5 s, sys: 12 ms, total: 20.5 s
Wall time: 20.4 s


### Logistic

In [42]:
%%time
ypred = classifier.predict(Xte)

CPU times: user 228 ms, sys: 0 ns, total: 228 ms
Wall time: 228 ms


In [43]:
pd.DataFrame(lb.inverse_transform(ypred),lb.inverse_transform(yte))

,0,1,2,3,4,5,6,7,8
"(usa, ussr)",ussr,None,None,None,None,None,None,None,None
"(usa,)",usa,None,None,None,None,None,None,None,None
"(canada, earn)",canada,earn,None,None,None,None,None,None,None
"(earn, usa)",earn,usa,None,None,None,None,None,None,None
"(china, trade, usa)",china,trade,usa,None,None,None,None,None,None
"(coffee, crude, kenya)",kenya,None,None,None,None,None,None,None,None
"(coffee, zimbabwe)",coffee,None,None,None,None,None,None,None,None
"(housing, usa)",usa,None,None,None,None,None,None,None,None
"(usa,)",usa,None,None,None,None,None,None,None,None
"(earn, usa)",earn,None,None,None,None,None,None,None,None


In [44]:
from sklearn.metrics import f1_score, precision_score,recall_score

precision = precision_score(yte, ypred, average='micro')
recall = recall_score(yte, ypred, average='micro')
f1 = f1_score(yte, ypred, average='micro')

print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

Micro-average quality numbers
Precision: 0.9164, Recall: 0.7868, F1-measure: 0.8467


In [45]:
precision = precision_score(yte, ypred, average='macro')
recall = recall_score(yte, ypred, average='macro')
f1 = f1_score(yte, ypred, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}"
        .format(precision, recall, f1))

Macro-average quality numbers
Precision: 0.4294, Recall: 0.3031, F1-measure: 0.3434


In [22]:
lb.inverse_transform(ypred)

[('usa',),
 ('usa',),
 ('usa',),
 ('acq', 'usa'),
 ('acq', 'usa'),
 ('usa',),
 ('acq', 'usa'),
 ('usa',),
 ('earn',),
 ('usa',),
 ('ship', 'usa'),
 ('netherlands',),
 ('grain', 'uk', 'usa', 'wheat'),
 ('canada', 'gnp'),
 ('earn', 'usa'),
 ('barley', 'grain', 'wheat'),
 ('usa',),
 ('interest', 'money-fx', 'usa'),
 ('earn', 'usa'),
 ('coffee', 'ico-coffee', 'usa'),
 ('grain', 'usa', 'wheat', 'yugoslavia'),
 ('acq', 'usa'),
 ('usa',),
 ('earn', 'usa'),
 ('switzerland',),
 ('canada', 'earn'),
 ('earn', 'usa'),
 ('usa',),
 ('usa',),
 ('usa',),
 ('canada',),
 ('belgium',),
 ('usa',),
 ('usa',),
 ('earn', 'usa'),
 ('earn', 'usa'),
 ('italy',),
 ('earn', 'usa'),
 ('italy',),
 ('usa',),
 ('usa',),
 ('earn', 'usa'),
 ('usa',),
 ('earn', 'usa'),
 ('trade', 'usa'),
 ('earn', 'usa'),
 ('acq', 'usa'),
 ('earn', 'usa'),
 ('usa',),
 ('usa', 'ussr'),
 ('usa',),
 ('usa',),
 ('imf', 'uganda', 'worldbank'),
 ('usa',),
 ('imf', 'jamaica', 'usa'),
 ('usa',),
 ('earn', 'usa'),
 ('usa',),
 ('acq', 'reagan'),
